## MSTICPy and Notebooks in InfoSec

---

<h1 style="border: solid; padding:5pt; color:black; background-color:#909090">Session 7 - Troubleshooting and Advanced Jupyter Topics</h1>

---

## What this session covers:

* Jupyter Kernels & Python environments
* Notebook magics
* Widgets introduction
* Troubleshooting and debugging notebooks


## Prerequisites
- Python >= 3.8 Environment
- Jupyter installed
- MSTICPy installed

## Recommended
- VS Code


---

# <a style="border: solid; padding:5pt; color:black; background-color:#909090">Jupyter Kernels & Python environments</a>

---

Python environments let you create "isolated" installations with independent versions of packages.

This is usually **A VERY GOOD IDEA**!


## <a style="border: solid; padding:5pt; color:black; background-color:#309030">Task 1 - Create a Python/Conda Environment</a>

1. Open a terminal (in VS Code/Anaconda prompt)
2. Enter the following commands (depending on your environment)

Conda

```
conda create -n firecon22
conda activate firecon22
conda install pip
```

Linux

```
python -m venv firecon22
source ./firecon22/Scripts/activate
```

Windows

```
python -m venv firecon22
.\firecon22\Scripts\activate
```




## Using different Python Kernels with Jupyter

> Note: VSCode seems to be able to use Python or Conda environments for notebooks without installing a kernel.<br>
> However, installing a dedicated ipykernel is needed for debugging.

## <a style="border: solid; padding:5pt; color:black; background-color:#309030">Task 2 - install an IPython kernel for your environment</a>

1. Install `ipykernel`
    ```
        python -m pip install ipykernel
    ```
    ....or
    ```
        conda install ipykernel
    ```

2. Create an IPython kernel for notebooks for your environment
    ```
        python -m ipykernel install --user --name firecon22 --display-name "Python3 (firecon22)"
    ```

---

## Addendum - removing unneeded kernels/environments

```
jupyter kernelspec remove KERNELNAME

```

Example

```
(base) e:\src\test>jupyter kernelspec list
Available kernels:
  firecon22              C:\Users\Ian\AppData\Roaming\jupyter\kernels\firecon22
  msticpy                C:\Users\Ian\AppData\Roaming\jupyter\kernels\msticpy
  py-jupyter-sentinel    C:\Users\Ian\AppData\Roaming\jupyter\kernels\py-jupyter-sentinel
  python3                C:\Users\Ian\AppData\Roaming\jupyter\kernels\python3
  bluehound              C:\ProgramData\jupyter\kernels\bluehoundon


(base) e:\src\test>jupyter kernelspec remove firecon22
Kernel specs to remove:
  firecon22         C:\Users\Ian\AppData\Roaming\jupyter\kernels\firecon22
Remove 1 kernel specs [y/N]: y
[RemoveKernelSpec] Removed C:\Users\Ian\AppData\Roaming\jupyter\kernels\firecon22
```

Remove the environment if you don't need it

Python venv - just delete the venv folder

Conda
```
conda remove --all -n MyNewCondaEnv
```

---

# <a style="border: solid; padding:5pt; color:black; background-color:#909090">Troubleshooting exceptions and debugging</a>

---

### A Python exception

In [1]:
# Bad code example

def bad_func(param1, param2):
    """What could possibly go wrong."""
    return param1 + param2

def func_in_middle(*args):
    """It's not my problem"""
    return bad_func(*args)

def hapless():
    """I'm just hoping for the best."""
    print(func_in_middle(1, 2))
    print(func_in_middle("Hello", "World"))
    print(func_in_middle("Hello", 1))


hapless()

3
HelloWorld


TypeError: can only concatenate str (not "int") to str

### Use %tb to review last traceback
<p style="font-family:consolas; font-size:15pt; color:green">
%tb
</p>


In [2]:
%tb

TypeError: can only concatenate str (not "int") to str

## <a style="border: solid; padding:5pt; color:black; background-color:#309030">Task 3 - Use `%xmode` magic to diagnose the error</a>

1. Turn on verbose traceback mode
2. Redisplay the traceback and identify the code error

Syntax:
<p style="font-family:consolas; font-size:15pt; color:green">
%mode { Verbose | Context | Plain | Minimal }
</p>

<details>
<summary>Hints...</summary>
<ul>
<li>Type the following into a cell and run to turn on parameter values.
<pre>%xmode verbose</pre> </li>
<li>Type the following into a cell to view the last error.
<pre>%tb</pre>
</li>
<ul>
</details>

In [3]:
%xmode verbose
hapless()

Exception reporting mode: Verbose
3
HelloWorld


TypeError: can only concatenate str (not "int") to str

## Exceptions within Exceptions

In [4]:
def func_in_middle2(*args):
    """It's not my problem but let me try to fix things"""
    try:
        return bad_func(*args)
    except TypeError as err:
        return "".join(str(arg.value) for arg in args)
    except Exception as err:
        raise RuntimeError("Something terrible happened") from err

def hapless2():
    """I'm just hoping for the best."""
    print(func_in_middle(1, 2))
    print(func_in_middle("Hello", "World"))
    print(func_in_middle2("Hello", 1))


hapless2()

3
HelloWorld


AttributeError: 'str' object has no attribute 'value'

## <a style="border: solid; padding:5pt; color:black; background-color:#309030">Task 4 - Debugging from VS Code</a>

If you are running in VS Code do one of the following:
- Hover mouse to the right of the line number and click on the red circle
- Put the cursor on the line and press F9
- Ctrl-Shift-P -> debug toggle breakpoint

Then
- Hover over execution arrow and select Debug
- Press Ctrl-Shift-Alt-Enter


In [5]:
hapless()

3
HelloWorld


TypeError: can only concatenate str (not "int") to str

## MSTICPy Exceptions


In [6]:
from msticpy.common.exceptions import MsticpyParameterError


def bad_func(param1, param2):
    """What could possibly go wrong."""
    if not isinstance(param1, int):
        raise MsticpyParameterError(
            "param1 must be an int",
            title="Bad param1",
            parameter=["param1"],
            help_uri=(
                "Dataproviders documentation",
                "https://msticpy.readthedocs.io/en/latest/data_acquisition/DataProviders.html",
            ),
        )
    return param1 + param2


def func_in_middle(*args):
    """It's not my problem"""
    return bad_func(*args)


def hapless():
    """I'm just hoping for the best."""
    print(func_in_middle(1, 2))
    print(func_in_middle("Hello", "World"))
    print(func_in_middle("Hello", 1))


hapless()

3


MsticpyParameterError: ('Bad param1', 'param1 must be an int', 'One or more parameters were incorrect.', 'param1', ('Dataproviders documentation', 'https://msticpy.readthedocs.io/en/latest/data_acquisition/DataProviders.html'))

In [ ]:
import msticpy as mp
mp.init_notebook()


In [8]:
hapless()

3


msticpy.common.exceptions.MsticpyParameterError('Bad param1',
                                                'param1 must be an int',
                                                'One or more parameters were incorrect.',
                                                'param1',
                                                ('Dataproviders documentation',
                                                 'https://msticpy.readthedocs.io/en/latest/data_acquisition/DataProviders.html'))

---

# <a style="border: solid; padding:5pt; color:black; background-color:#909090">Logging/Tracing</a>

---

- Use the `verbosity` parameter with `init_notebook` to understand what's going on during startup

In [1]:
import msticpy as mp
mp.init_notebook(verbosity=2)

## Python logging

By default this is set to "WARNING"

- Use `mp.set_logging_level("INFO") to see more detailed output


<div style="border: solid; padding: 5pt"><b>Note:</b>
We've only recently started instrumenting logging across MSTICPy
so not all areas are covered.
</div>

Things with good coverage:
- Authentication
- QueryProvider (especially newer ones)
- `init_notebook`


In [2]:
mp.set_logging_level("INFO")
mp.init_notebook()

2023-06-24 18:11:32,393: INFO - Starting Notebook initialization (nbinit#413)
2023-06-24 18:11:32,821: INFO - msticpy version installed: 2.5.3 latest published: 2.5.3
Latest known version is installed.
 (nbinit#424)
2023-06-24 18:11:32,824: INFO - Imported:pd (pandas), IPython.get_ipython, IPython.display.display, IPython.display.HTML, IPython.display.Markdown, widgets (ipywidgets), pathlib.Path, np (numpy), msticpy, msticpy.data.QueryProvider, msticpy.vis.foliummap.FoliumMap, msticpy.context.TILookup, msticpy.context.GeoLiteLookup, msticpy.context.IPStackLookup, msticpy.transform.IoCExtract, msticpy.common.utility.md, msticpy.common.utility.md_warn, msticpy.common.wsconfig.WorkspaceConfig, msticpy.init.pivot.Pivot, msticpy.datamodel.entities, msticpy.init.nbmagics, msticpy.nbtools.SecurityAlert, msticpy.vis.mp_pandas_plot, msticpy.vis.nbdisplay, msticpy.init.mp_pandas_accessors, msticpy.nbwidgets (nbinit#620)
2023-06-24 18:11:32,825: INFO -  (nbinit#441)
2023-06-24 18:11:32,829: INFO 

In [4]:
qry_prov = mp.QueryProvider("MSSentinel_New")
qry_prov.connect()

2023-06-24 18:12:29,966: INFO - AzureMonitorDriver loaded. connect_str  None, kwargs: {'data_environment': <DataEnvironment.MSSentinel_New: 16>} (azure_monitor_driver#146)
2023-06-24 18:12:29,997: INFO - Read 5 queries from F:\anaconda\envs\firecon23\Lib\site-packages\msticpy\data\queries\mssentinel\kql_sent_alert.yaml (data_query_reader#83)
2023-06-24 18:12:30,043: INFO - Read 10 queries from F:\anaconda\envs\firecon23\Lib\site-packages\msticpy\data\queries\mssentinel\kql_sent_azure.yaml (data_query_reader#83)
2023-06-24 18:12:30,101: INFO - Read 8 queries from F:\anaconda\envs\firecon23\Lib\site-packages\msticpy\data\queries\mssentinel\kql_sent_azuresentinel.yaml (data_query_reader#83)
2023-06-24 18:12:30,120: INFO - Read 3 queries from F:\anaconda\envs\firecon23\Lib\site-packages\msticpy\data\queries\mssentinel\kql_sent_az_dns.yaml (data_query_reader#83)
2023-06-24 18:12:30,143: INFO - Read 1 queries from F:\anaconda\envs\firecon23\Lib\site-packages\msticpy\data\queries\mssentinel\k

connected


---

# <a style="border: solid; padding:5pt; color:black; background-color:#909090">Additional Material</a>

---

---

# <a style="border: solid; padding:5pt; color:black; background-color:#909090">IPython Magics</a>

---

[https://ipython.readthedocs.io/en/stable/interactive/magics.html](https://ipython.readthedocs.io/en/stable/interactive/magics.html)

## What are they?

Magics are a kind of macro/function that allows you to invoke functionality
of the notebook or OS independent of the kernel language.

### Line magics - single %
- Only operate on the arguments on the remainder of the line
- Can be mixed with other code

### Cell magics - double %%
- Operate on whole cell contents
- Must be in their own cell and at the start of the cell (even comments!)

## Popular magics - 
<p style="font-family:consolas; font-size:15pt; color:green">
%magic %env %readfile %writefile %run %hmtl %pip %logstart 
</p>

**%magic** - lists all magic functions (LONG!)

**%logstart** log_file - very useful if you are prone to deleting/overwriting your code and then regret it

**%env** - get or set an environment variable

**%tb** and **%xmode** covered in later section

**%pdb** - if you are not running in VSCode or Jupyterlab

## Invoking shell commands

Prefix with !

These are not magics - they directly invoke underlying OS commands.

Like line magics, can use these mixed with other code

In [9]:
!dir

 Volume in drive E has no label.
 Volume Serial Number is 7E50-19F7

 Directory of e:\src\msticpy-training\workshops\Jun2023\completed

2023-06-26  17:10    <DIR>          .
2023-06-26  16:10    <DIR>          ..
2023-06-26  14:15         2,024,479 Session-1-IntroToMsticpy-complete.ipynb
2023-06-24  18:07            36,454 Session-2-MSTICPy-Configuration-complete.ipynb
2023-06-26  16:56           342,362 Session-3-AcquiringData-complete.ipynb
2023-06-26  16:09        19,321,557 Session-4-Visualization-complete.ipynb
2023-06-26  16:46           505,000 Session-5-Enrichment-complete.ipynb
2023-06-26  17:09           110,183 Session-6-Data-Analysis-complete.ipynb
2023-06-26  17:13            64,030 Session-7-Troubleshooting and Advanced Topics.ipynb
2022-10-26  12:41             4,000 Session-8-AutomatedNotebooks.ipynb
               8 File(s)     22,408,065 bytes
               2 Dir(s)  156,314,730,496 bytes free


In [10]:
# Use a shell command to get current directory files
cd_files = !dir

# If the line contains the word "Session", split out the filename
# and save to a list
notebooks = [
    ''.join(line.split()[3:])
    for line in cd_files
    if "Session" in line
]

print(notebooks)

['Session-1-IntroToMsticpy-complete.ipynb', 'Session-2-MSTICPy-Configuration-complete.ipynb', 'Session-3-AcquiringData-complete.ipynb', 'Session-4-Visualization-complete.ipynb', 'Session-5-Enrichment-complete.ipynb', 'Session-6-Data-Analysis-complete.ipynb', 'Session-7-TroubleshootingandAdvancedTopics.ipynb', 'Session-8-AutomatedNotebooks.ipynb']


---

# <a style="border: solid; padding:5pt; color:black; background-color:#909090">Notebook Widgets</a>

---

Get input in a controllable way

### Text Widget

In [11]:
import ipywidgets as widgets

txt = widgets.Text(description="Type something long")
display(txt)

Text(value='', description='Type something long')

Use widget `Layout` and `style` to make things look better

In [12]:
txt = widgets.Text(
    description="Type something long",
    layout=widgets.Layout(width="50%", padding="5pt"),
    style={"description_width": "initial"}
)
display(txt)

# You can re-use the style/layout by saving to a dict variable.
WIDGET_DEFAULTS = {
    "layout": widgets.Layout(width="50%", padding="5pt"),
    "style": {"description_width": "initial"},
}
txt2 = widgets.Text(description="Type something extremely long", **WIDGET_DEFAULTS)
display(txt2)

Text(value='', description='Type something long', layout=Layout(padding='5pt', width='50%'), style=TextStyle(d…

Text(value='', description='Type something extremely long', layout=Layout(padding='5pt', width='50%'), style=T…

### Select Widget

In [13]:
sm = widgets.Select(
    options=['Option1', 'Option2', 'Option3'],
    # rows=10,
    description='Modules',
    disabled=False,
    **WIDGET_DEFAULTS,
)
display(sm)

Select(description='Modules', layout=Layout(padding='5pt', width='50%'), options=('Option1', 'Option2', 'Optio…

In [15]:
sm.value

'Option2'

## <a style="border: solid; padding:5pt; color:black; background-color:#309030">Task 5 - Create a selection list of notebooks</a>

Use the code examples to build a Selection list of Session notebooks in the current directory.
1. Get a list of notebook titles from the current directory
2. Create an option list to select a notebook name.

<details>
<summary>Hints...</summary>
<ul>
<li>Use Python list `notebooks` from the <b>Invoking shell commands</b> section</li>
<li>Use a `widgets.Select` control</li>
<li>Pass `notebooks` as the `options` parameter</li>
<li>Bonus points for making it look pretty</li>
</ul>
</details>

In [16]:
nb_select = widgets.Select(
    options=notebooks,
    # rows=10,
    description='Notebooks',
    disabled=False,
    **WIDGET_DEFAULTS,
)
nb_select

nb_select # display the widget

Select(description='Notebooks', layout=Layout(padding='5pt', width='50%'), options=('Session-1-IntroToMsticpy-…

In [17]:
# test
assert "Session" in nb_select.value

### And there are a lot more widgets...

In [18]:
from datetime import datetime
w_date = widgets.DatePicker(
    description='Pick a Date',
    disabled=False,
    value=datetime.utcnow(),
)
w_multi = widgets.SelectMultiple(description="MultiSelect", options=sm.options, value=["Option1", "Option3"])
w_slide = widgets.IntSlider(description="Int value", value=42)

# Put them all in a horizontal box
widgets.HBox([w_date, w_multi, w_slide], layout=widgets.Layout(padding="10pt"))

### MSTICPy also includes a number of composite widgets.

- QueryTime
- QueryBrowser
- MultiSelect

In [19]:
import msticpy as mp
mp.nbwidgets.QueryTime()

---
# End of Session
